In [22]:
from zenml import step, pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np
from scipy.special import softmax
import urllib.request
import csv
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import mlflow
import pandas as pd
import great_expectations as ge

label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}


@step
def read_tweets_from_file(file_path: str) -> list:
    with open(file_path, 'r', encoding='utf-8') as file:
        tweets = file.readlines()
    return [tweet.strip() for tweet in tweets]

@step
def read_labels_from_file(labels_path: str) -> list:
    with open(labels_path, 'r', encoding='utf-8') as file:
        labels = [int(line.strip()) for line in file]
    return labels

@step
def preprocess_step(texts: list) -> list:
    preprocessed_texts = []
    for text in texts:
        new_text = []
        for t in text.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        preprocessed_texts.append(" ".join(new_text))
    return preprocessed_texts


In [23]:
from zenml.materializers.base_materializer import BaseMaterializer
from transformers import AutoModelForSequenceClassification

@step
def model_inference_step(texts: list) -> list:
    predictions = []
    task = 'sentiment'
    MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)

    mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
    with urllib.request.urlopen(mapping_link) as f:
        html = f.read().decode('utf-8').split("\n")
        csvreader = csv.reader(html, delimiter='\t')
        labels = [row[1] for row in csvreader if len(row) > 1]

    for text in texts:
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)[::-1]
        text_predictions = [labels[i] for i in ranking] 
        predictions.append(text_predictions[0])
    return predictions


In [24]:
@step
def evaluate_predictions(predictions: list, true_labels: list) -> dict:
    predictions_mapped = [label_mapping[pred] for pred in predictions]
    
    accuracy = accuracy_score(true_labels, predictions_mapped)
    precision = precision_score(true_labels, predictions_mapped, average='weighted', zero_division=0)
    recall = recall_score(true_labels, predictions_mapped, average='weighted', zero_division=0)
    f1 = f1_score(true_labels, predictions_mapped, average='weighted', zero_division=0)
    
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [25]:
import matplotlib.pyplot as plt

@step
def visualize_metrics(metrics: dict) -> str:
    
    names = list(metrics.keys())
    values = list(metrics.values())
    
    plt.figure(figsize=(10, 5))
    plt.bar(names, values)
    plt.ylabel('Score')
    plt.title('Model Evaluation Metrics')
    
    figure_path = 'metrics_figure.png'
    plt.savefig(figure_path)
    plt.close()
    
    return figure_path

In [26]:
@pipeline
def sentiment_analysis_pipeline_with_evaluation(file_path: str, labels_path: str):
    tweets = read_tweets_from_file(file_path)
    true_labels = read_labels_from_file(labels_path)
    processed_texts = preprocess_step(tweets)
    predictions = model_inference_step(processed_texts)
    evaluation_results = evaluate_predictions(predictions, true_labels)
    figure_path = visualize_metrics(evaluation_results)

if __name__ == "__main__":
    file_path = '../val_text.txt' 
    labels_path = '../val_labels.txt' 
    sentiment_analysis_pipeline_with_evaluation(file_path, labels_path)

Initiating a new run for the pipeline: sentiment_analysis_pipeline_with_evaluation.
Reusing registered pipeline version: (version: 20).
Executing a new run.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
Using cached version of read_labels_from_file.
Step read_labels_from_file has started.
Using cached version of read_tweets_from_file.
Step read_tweets_from_file has started.
Using cached version of preprocess_step.
Step preprocess_step has started.
Using cached version of model_inference_step.
Step model_inference_step has started.
Using cached version of evaluate_predictions.
Step evaluate_predictions has started.
Using cached version of visualize_metrics.
Step visualize_metrics has started.
Pipeline run has finished in 1.111s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


In [27]:







from zenml.client import Client

artifact = Client().get_artifact_version('60fe71ea-01a7-4f80-8f91-29a4821cfeec')
loaded_artifact = artifact.load()





print(loaded_artifact)



['neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'negative', 'positive', 'neutral', 'negative', 'positive', 'neutral', 'neutral', 'negative', 'positive', 'positive', 'neutral', 'positive', 'negative', 'positive', 'negative', 'neutral', 'positive', 'negative', 'positive', 'positive', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'positive', 'positive', 'negative', 'neutral', 'neutral', 'negative', 'positive', 'positive', 'negative', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'positive', 'positive', 'positive', 'neutral', 'neutral', 'neutral', 'negative', 'positive', 'neutral', 'negative', 'neutral', 'positive', 'positive', 'neutral', 'neutral', 'positive', 'neutral', 'positive', 'positive', 'negative', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'negative', 'positi